In [42]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [43]:
appearances_train = pd.read_csv("appearances_first_3_days.csv", header=None, names=["device_id", "date", "seconds_until_next_appearence"])

In [44]:
#appearances_test =  pd.read_csv("appearances_second_3_days.csv", header=None, names=["device_id", "date", "seconds_until_next_appearence"])

In [45]:
#convertions_train = pd.read_csv("auctions_deltas_to_convertion_train.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [46]:
#convertions_test = pd.read_csv("auctions_deltas_to_convertion_test.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [47]:
ids_to_predict = pd.read_csv("target_competencia_ids.csv")

In [48]:
ids_to_predict = ids_to_predict[["ref_hash"]]

In [49]:
appearances_train["date"] = pd.to_datetime(appearances_train["date"])

In [50]:
#appearances_test["date"] = pd.to_datetime(appearances_test["date"])

In [51]:
#convertions_test["date"] = pd.to_datetime(convertions_test["date"])

In [52]:
#convertions_train["date"] = pd.to_datetime(convertions_train["date"])

In [53]:
events = pd.read_csv("events.csv")

/home/martinrosas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
#appearances_test["is_weekend"] = appearances_test.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [55]:
#convertions_test["is_weekend"] = convertions_test.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [56]:
#convertions_train["is_weekend"] = convertions_train.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [57]:
ids_to_predict_numbers = ids_to_predict.copy()

In [58]:
ids_to_predict_numbers["ref_hash"] = ids_to_predict_numbers["ref_hash"].apply(lambda string: int(string.split("_")[0]))

In [59]:
ids_to_predict_numbers = ids_to_predict_numbers.drop_duplicates()

In [60]:
ids_to_predict_numbers["date"] = datetime(2019, 5, 27, 0,0,0)

In [61]:
ids_to_predict_numbers.head()

,ref_hash,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [62]:
ids_to_predict_numbers_appearances = ids_to_predict_numbers.copy()

In [63]:
ids_to_predict_numbers_convertion = ids_to_predict_numbers.copy()

In [64]:
#voy a aplicar mismo cambios a ids_to_predict_numbers_appearances y a appearances_train para probar que onda

In [65]:
appearances_train.head()

,device_id,date,seconds_until_next_appearence
0,4172466725848941608,2019-04-18 06:27:50.996158,848.908411
1,4172466725848941608,2019-04-18 06:41:59.904569,28513.038578
2,4172466725848941608,2019-04-18 14:37:12.943147,199.094384
3,4172466725848941608,2019-04-18 14:40:32.037531,185.775504
4,4172466725848941608,2019-04-18 14:43:37.813035,255.313251


In [66]:
ids_to_predict_numbers_appearances.head()

,ref_hash,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [67]:
ids_to_predict_numbers_appearances = ids_to_predict_numbers_appearances.rename(columns={"ref_hash": "device_id"})

In [68]:
ids_to_predict_numbers_appearances.head()

,device_id,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [69]:
#les agrego los features

In [70]:
#appearances_train["is_weekend"] = appearances_train["data"].apply(lambda date: date.weekday()>4)

In [71]:
#ids_to_predict_numbers_appearances["is_weekend"] = ids_to_predict_numbers_appearances["date"].apply(lambda date: date["date"].weekday()>4)

In [72]:
most_used_app = events.groupby("ref_hash")["application_id"].agg(lambda x: x.mode()).rename_axis("device_id").to_frame().reset_index()


In [135]:
#arreglo que algunos casos la app id mas usada es una lista. porque la funcion .mode() devolvio mas de un elemento
most_used_app["application_id"] = most_used_app["application_id"].apply(lambda x: x[0] if type(x) == np.ndarray else x)

In [136]:
df_train = appearances_train.merge(most_used_app, on="device_id", how="left")

In [137]:
df_test = ids_to_predict_numbers_appearances.merge(most_used_app, on="device_id", how="left")

In [138]:
#renombro para dejar mas claro que el nombre del nuevo feature
df_train = df_train.rename({"application_id": "most_used_app"}, axis="columns")
df_test = df_test.rename({"application_id": "most_used_app"}, axis="columns")

In [139]:
#reemplazo los nan de most used app por un nuemero no existente
df_train["most_used_app"] = df_train["most_used_app"].fillna(-1)
df_test["most_used_app"] = df_test["most_used_app"].fillna(-1)

In [140]:
def is_android_or_macos(row):
    if row["user_agent"] is np.nan:
        return None
    agent = row["user_agent"].lower()
    if "android" in agent:
        return "android"
    elif "darwin" in agent:
        return "mac os"
    else:
        return "other"

In [141]:
installs = pd.read_csv("installs.csv")

/home/martinrosas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [142]:
installs["os"] = installs.apply(is_android_or_macos, axis=1)

In [143]:
os = installs[["ref_hash", "os"]].dropna()

In [144]:
#saco ids duplicados
os = os.drop_duplicates(subset="ref_hash", keep='first').rename({"ref_hash": "device_id"},axis="columns")

In [145]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(os,how="left", on="device_id")
df_test = df_test.merge(os,how="left", on="device_id")

In [146]:
df_train["os"] = df_train["os"].fillna("other")
df_test["os"] = df_test["os"].fillna("other")

In [147]:
#PRUEBO OTRA VEZ con random forest
#antes tengo que convertir a numeros los campos que son string (en este caso solo OS)
#USO LABEL ENCODER
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [148]:
#encodeo columna os
le.fit(df_train["os"])
df_train["os"] = le.transform(df_train["os"])
df_test["os"] = le.transform(df_test["os"])

In [149]:
#ahora en vez de string tengo sus codigos
df_train["os"].head()

0    2
1    2
2    2
3    2
4    2
Name: os, dtype: int64

In [151]:

#CREO X_TRAIN Y Y_TRAIN SIN DEVICE ID
X_train = df_train[["device_id",
       'most_used_app', 'os']]
y_train = df_train['seconds_until_next_appearence']
X_test = df_test[["device_id",
       'most_used_app', 'os']]

In [102]:

from sklearn.ensemble import RandomForestRegressor

In [153]:
regr = RandomForestRegressor()
#entreno el modeo
regr.fit(X_train, y_train)  
y_pred = regr.predict(X_test)

/home/martinrosas/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [154]:
prediction_appearances = X_test[["device_id"]]
prediction_appearances["seconds_until_next_appearance"] = y_pred

In [155]:
prediction_appearances.head()

,device_id,most_used_app,os,seconds_until_next_appearance
0,1000169251625791246,21.0,2,18815.842080
1,1000395625957344683,121.0,0,1656.922180
2,1003027494996471685,75.0,2,15000.126804
3,1006670001679961544,68.0,1,14251.089592
4,1007573308966476713,94.0,2,31241.971958


In [112]:
#ahora hago la prediccion para tiempo en instalar

In [114]:
ids_to_predict_numbers_convertion = ids_to_predict_numbers_convertion.rename(columns={"ref_hash": "device_id"})

In [115]:
ids_to_predict_numbers_convertion.head()

,device_id,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [109]:
convertions_train = pd.read_csv("auctions_deltas_to_convertion_train.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [111]:
convertions_train.head()

,device_id,date,seconds_until_convertion
0,8323952185725734415,2019-04-20 05:35:45.651502,259200.0
1,8323952185725734415,2019-04-20 05:40:46.744391,259200.0
2,8323952185725734415,2019-04-20 05:48:22.167361,259200.0
3,8323952185725734415,2019-04-20 05:51:43.749231,259200.0
4,8323952185725734415,2019-04-20 05:51:52.533312,259200.0


In [ ]:
#agrego mismos features

In [116]:
df_train = convertions_train.merge(most_used_app, on="device_id", how="left")

In [117]:
df_test = ids_to_predict_numbers_convertion.merge(most_used_app, on="device_id", how="left")

In [118]:
#renombro para dejar mas claro que el nombre del nuevo feature
df_train = df_train.rename({"application_id": "most_used_app"}, axis="columns")
df_test = df_test.rename({"application_id": "most_used_app"}, axis="columns")

In [119]:
#reemplazo los nan de most used app por un nuemero no existente
df_train["most_used_app"] = df_train["most_used_app"].fillna(-1)
df_test["most_used_app"] = df_test["most_used_app"].fillna(-1)

In [120]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(os,how="left", on="device_id")
df_test = df_test.merge(os,how="left", on="device_id")

In [126]:
df_train["os"] = df_train["os"].fillna("other")
df_test["os"] = df_test["os"].fillna("other")

In [121]:
#PRUEBO OTRA VEZ con random forest
#antes tengo que convertir a numeros los campos que son string (en este caso solo OS)
#USO LABEL ENCODER
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [127]:
#encodeo columna os
le.fit(df_train["os"])
df_train["os"] = le.transform(df_train["os"])
df_test["os"] = le.transform(df_test["os"])

In [130]:

#CREO X_TRAIN Y Y_TRAIN SIN DEVICE ID
X_train = df_train[["device_id",
       'most_used_app', 'os']]
y_train = df_train['seconds_until_convertion']
X_test = df_test[["device_id",
       'most_used_app', 'os']]

In [131]:
regr = RandomForestRegressor()
#entreno el modeo
regr.fit(X_train, y_train)  
y_pred = regr.predict(X_test)

/home/martinrosas/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [132]:
prediction_convertion = X_test[["device_id"]]
prediction_convertion["seconds_until_convertion"] = y_pred

In [133]:
prediction_appearances.head()

,most_used_app,os,seconds_until_next_appearance
0,21.0,2,1889.356538
1,121.0,0,1967.793420
2,75.0,2,2341.442581
3,68.0,1,2201.501866
4,94.0,2,2816.460681


In [134]:
prediction_convertion.head()

,device_id,most_used_app,os,seconds_until_next_appearance
0,1000169251625791246,21.0,2,259200.000000
1,1000395625957344683,121.0,0,91823.809995
2,1003027494996471685,75.0,2,259200.000000
3,1006670001679961544,68.0,1,239620.514607
4,1007573308966476713,94.0,2,259200.000000


In [156]:
#ahora paso al formato de submit

In [174]:
prediction_appearances = prediction_appearances.rename(columns={"device_id": "ref_hash", "seconds_until_next_appearance": "obj"})

In [173]:
prediction_convertion = prediction_convertion.rename(columns={"device_id": "ref_hash", "seconds_until_convertion": "obj"})

In [168]:
prediction_appearances["ref_hash"] = prediction_appearances["ref_hash"].apply(lambda ref: str(ref)+"_st")

In [170]:
prediction_convertion["ref_hash"] = prediction_convertion["ref_hash"].apply(lambda ref: str(ref)+"_sc")

In [175]:
prediction_appearances.head()

,ref_hash,Predicted_Value
0,1000169251625791246_st,18815.842080
1,1000395625957344683_st,1656.922180
2,1003027494996471685_st,15000.126804
3,1006670001679961544_st,14251.089592
4,1007573308966476713_st,31241.971958


In [176]:
prediction_convertion.head()

,ref_hash,Predicted_Value
0,1000169251625791246_sc,259200.000000
1,1000395625957344683_sc,91823.809995
2,1003027494996471685_sc,259200.000000
3,1006670001679961544_sc,239620.514607
4,1007573308966476713_sc,259200.000000


In [177]:
#concateno las predicciones porque tienen que devolverse todas juntas

In [181]:
full_pred = pd.concat([prediction_appearances, prediction_convertion])

In [ ]:
#hago un left join con ids to predict para oredenarlos como pide el enunciado

In [182]:
ids_to_predict.head()

,ref_hash
0,1000169251625791246_sc
1,1000169251625791246_st
2,1000395625957344683_sc
3,1000395625957344683_st
4,1003027494996471685_sc


In [184]:
full_pred = ids_to_predict.merge(full_pred, on="ref_hash", how="left")

In [192]:
full_pred.head()

Index(['ref_hash', 'obj'], dtype='object')

In [200]:
full_pred["obj"] = full_pred["obj"].astype(np.int64)

In [203]:
full_pred.to_csv("first_pred.csv", index=False)

In [196]:
full_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8074 entries, 0 to 8073
Data columns (total 2 columns):
ref_hash    8074 non-null object
obj         8074 non-null float64
dtypes: float64(1), object(1)
memory usage: 189.2+ KB


In [197]:
pd.read_csv("target_competencia_ids.csv").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8074 entries, 0 to 8073
Data columns (total 2 columns):
ref_hash    8074 non-null object
obj         8074 non-null int64
dtypes: int64(1), object(1)
memory usage: 126.2+ KB
